Install the tweepy library

In [1]:
# !pip install tweepy
# !pip install mysqlclient

Import required libraries

In [ ]:
import os
import re
import time

import tweepy as tw
import pandas as pd
from datetime import datetime, timedelta, timezone
import MySQLdb


# For sentiment analysis of tweets
from textblob import TextBlob

Create Tweeter Consumer Keys and Access Tokens 

In [ ]:
consumer_key= 'NFIO7bIgwQpdCZ7p7rgewKXss'
consumer_secret= '2QegNrIpcGCW6nCWFTL5JEsrDCmYZpQHJ8SdIT7V0ql5cQgFwV'
access_token= '3049227557-36ln3qak9xbxHvgNos45cf3uScGxuUdaDAR4vqs'
access_token_secret= 'pZ4Y0p31graKrAYAy41mYztvIrYFzof7awiTDWQr2ui9u'

Initialize AuthHandler

In [ ]:
auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth, wait_on_rate_limit=True)

Define search words and date

In [ ]:
search_words = "#Covid"
date_since = "2018-11-16"

In [ ]:
tweets = tw.Cursor(api.search, q=search_words, lang="en", since=date_since).items(5)

In [ ]:
tweets

Iterate through tweets using for loop

In [ ]:
from datetime import datetime, timedelta
from email.utils import parsedate_tz

s='Mon Dec 07 17:03:03 +0000 2020'
def to_datetime(datestring):
    time_tuple = parsedate_tz(datestring.strip())
    dt = datetime(*time_tuple[:6])
    return dt - timedelta(seconds=time_tuple[-1])

print(to_datetime(s))
print(to_datetime(s).time().strftime("%H:%M:%S"))

x=datetime.strptime('12/31/13 00:12', "%m/%j/%y %H:%M").time().strftime("%H:%M:%S")

print(x)

Retrieve tweets as a collection of list

In [ ]:
[tweet.text for tweet in tweets]

Search Tweets without Retweets

In [ ]:
new_search = search_words + " -filter:retweets"

In [ ]:
new_search

In [ ]:
tweets = tw.Cursor(api.search, q=new_search, lang="en",since=date_since).items(5)

In [ ]:
[tweet.text for tweet in tweets]

Get username and location

In [ ]:
tweets = tw.Cursor(api.search,q=new_search, lang="en", since=date_since).items(5)

In [ ]:
users_locs = [[tweet.user.screen_name,tweet.user.location,tweet.text] for tweet in tweets]

In [ ]:
users_locs

Create a Pandas Dataframe From A List of Tweet Data

In [ ]:
tweet_df = pd.DataFrame(data=users_locs, columns=['user', "location","tweet"])

In [ ]:
tweet_df